<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from esper.prelude import *
from datetime import datetime
import scannertools as st
import traceback

In [ ]:
st.init_storage(bucket=os.environ['BUCKET'])

In [ ]:
files = sp.check_output('gsutil ls gs://{}/tvnews/videos/*.mp4'.format(os.environ['BUCKET']), shell=True).decode('utf-8')

In [ ]:
files.split('\n')[4]

In [ ]:
channels = {c.name: c for c in Channel.objects.all()}
shows = {c.name: c for c in Show.objects.all()}

In [ ]:
def gen_video(path):
    try:
        path = path.replace('gs://{}/'.format(os.environ['BUCKET']), '')
        item_name = os.path.splitext(path.split('/')[-1])[0]
        parts = item_name.split('_')
        [channel, date_str, time_str] = parts[:3]

        # Remove trailing W from channel, e.g. CNNW -> CNN
        if channel[-1] == 'W':
            channel = channel[:-1]

        show = ' '.join(parts[3:])
        
        date = datetime.strptime('{} {} UTC'.format(date_str, time_str), '%Y%m%d %H%M%S %Z')
        stv = st.Video(path)
        return Video(
            path=path,        
            num_frames=stv.num_frames(),
            fps=stv.fps(),
            width=stv.width(),
            height=stv.height(),
            channel=channels[channel],
            show=shows[show],
            time=date        
        )
    except:
        print(path)
        traceback.print_exc()


videos = par_for(gen_video, files.strip().split('\n'), workers=24)

In [ ]:
import django.utils.timezone as tz
def p(v):
    print('{}: {}'.format(v.path, v.time))
with tz.override('UTC'):
    p(Video.objects.all()[0])    
    p(Video.objects.all()[1])

In [ ]:
videos[0]

In [ ]:
Video.objects.filter(id=62047)

In [ ]:
current_paths = set([v['path'] for v in Video.objects.all().values('path')])

In [ ]:
new_videos = [v for v in videos[1:] if v.path not in current_paths]

In [ ]:
Video.objects.bulk_create(new_videos)

In [ ]:
Video.objects.all().count()

In [ ]:
# for s in tqdm(new_shows):
#     if s not in shows:
#         cs = CanonicalShow(name=s)
#         cs.save()
#         shows[s] = Show(name=s, canonical_show=cs)
#         shows[s].save()